In [47]:
import pandas as pd
import numpy as np
import os
import glob as gb
import sys
#from sqlalchemy import create_engine
sys.path.insert(0, "../../")
from clases.bd.conexion2 import MyDatabase2
conn = MyDatabase2() 
anio=2024

sql="""SELECT c.nu_doc_madre FROM maestros.cnv c WHERE  EXTRACT(YEAR FROM c.fe_nacido) = %s"""% (anio)
cnv=conn.df(sql) 

his=f"""
    SELECT a.id_tipo_documento, a.numero_documento, a.semana, a.id_establecimiento
    FROM (
        SELECT DISTINCT
            mp.id_tipo_documento,
            mp.numero_documento,
            ROUND((current_date - nt.fecha_ultima_regla) / 7) AS semana,
            nt.id_establecimiento,
            ROW_NUMBER() OVER (PARTITION BY mp.id_tipo_documento, mp.numero_documento, nt.anio ORDER BY nt.fecha_atencion DESC) AS rn
        FROM maestros.nominal_trama nt
        LEFT JOIN maestros.maestro_paciente mp ON mp.id_paciente = nt.id_paciente 
        WHERE (nt.codigo_item LIKE ANY (ARRAY['Z349%', 'Z359%']) OR nt.id_otra_condicion = 1) AND nt.anio = {anio}
    ) AS a
    WHERE a.rn = 1 AND a.semana < 45
""" 
#print(his)
his=conn.df(his)


#df1_sin_df2 = his[~his.isin(cnv.to_dict(orient='list')).all(axis=1)]
#df = df1_sin_df2.dropna()

# Selecciona solo las columnas necesarias de los DataFrames
#df1_sin_df2 = his[~his["numero_documento"].isin(cnv["nu_doc_madre"])]
#df = df1_sin_df2.dropna()

his["st"] = his["numero_documento"].isin(cnv["nu_doc_madre"]).astype(int)
his.fillna(0, inplace=True)
# Imprime el DataFrame his
print(his)

Conexion exitosa con la Base de datos:irvin_hisminsa

    SELECT a.id_tipo_documento, a.numero_documento, a.semana, a.id_establecimiento
    FROM (
        SELECT DISTINCT
            mp.id_tipo_documento,
            mp.numero_documento,
            ROUND((current_date - nt.fecha_ultima_regla) / 7) AS semana,
            nt.id_establecimiento,
            ROW_NUMBER() OVER (PARTITION BY mp.id_tipo_documento, mp.numero_documento, nt.anio ORDER BY nt.fecha_atencion DESC) AS rn
        FROM maestros.nominal_trama nt
        LEFT JOIN maestros.maestro_paciente mp ON mp.id_paciente = nt.id_paciente 
        WHERE (nt.codigo_item LIKE ANY (ARRAY['Z349%', 'Z359%']) OR nt.id_otra_condicion = 1) AND nt.anio = 2023
    ) AS a
    WHERE a.rn = 1 AND a.semana < 45

      id_tipo_documento numero_documento  semana  id_establecimiento  st
0                     1         40731760    19.0                2383   0
1                     1         40703233    27.0                2314   0
2               

In [51]:

conn.insert_df(his,'gestantes','materno')
his=conn.sql("CREATE INDEX gestantes_numero_documento_idx ON materno.gestantes (numero_documento,id_tipo_documento);")

#df_sin_duplicados.to_sql(table_name, engine,schema='maestros', if_exists='replace', index=False)

DataFrame insertado en la tabla gestantes


buscar gestantes 

In [4]:
# Convierte los DNIs a una lista
dnis = df['numero_documento'].tolist()
# Convierte los DNIs en una cadena separada por comas para usar en la consulta SQL
numeros_dni_str = ','.join(["'" + str(dni) + "'" for dni in dnis])

# Crea la consulta SQL con una cláusula WHERE para filtrar por los DNIs
# Consulta SQL con los números de DNI
sql = f"SELECT nt.*  FROM maestros.nominal_trama2 nt "\
      f"INNER JOIN maestros.maestro_paciente mp ON mp.id_paciente =nt.id_paciente  " \
      f"WHERE mp.numero_documento IN ({numeros_dni_str}) and nt.anio=2023"
print(sql)
gestantes=conn.df(sql)
#print(sql)


SELECT nt.*  FROM maestros.nominal_trama2 nt INNER JOIN maestros.maestro_paciente mp ON mp.id_paciente =nt.id_paciente  WHERE mp.numero_documento IN ('41396452','41917626','42140092','41272708','23979987','41597138','41019632','40882253','40778597','41895302','40737607','41949518','41398930','41148687','25012583','41145439','41830067','25324117','40591675','41727423','41911368','40647198','41139046','40886156','45277134','77792635','60572374','62215156','46644189','74572549','76134272','45080743','46558310','45318401','61243394','43812751','44595060','73648436','75822412','48918683','71803050','48371251','73117821','43262315','75944205','77041319','76690605','73431519','70461959','78008963','71964789','43903839','75872115','74277420','77542501','44530739','76353996','76072411','73600550','76381200','76465079','74245374','72169501','71794690','77138645','70080768','73299683','73364769','48451484','47190931','45875532','75749714','77166112','78716600','48332057','47363410','48248132','74

d:\Irvin\Irvin\Python\Python-GERESA\modulos\gestante\../..\clases\bd\conexion2.py:42: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  s = pd.read_sql(sql, con=self.conn)


KeyboardInterrupt: 